# Gradio UI 

In [19]:
# imports

import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
import google.generativeai as genai
import ollama

### Basic function for API call to llm

In [20]:
load_dotenv(override=True)
api_key = os.getenv('GEMINI_API_KEY')

# connect to gemini
genai.configure(api_key = api_key)

In [21]:
system_prompt = "You are a helpful assistant"
gemini_model = "gemini-1.5-flash"
ollama_model = "llama3.2"

model = genai.GenerativeModel(gemini_model, system_instruction=system_prompt)

In [7]:
def call_gemini(prompt):
    response = model.generate_content(prompt)
    return response.text

In [8]:
call_gemini("What's date today?")

'Today is October 26, 2023.\n'

### Gradio UI starter

In [9]:
import gradio as gr 

In [ ]:
gr.Interface(fn=call_gemini, inputs="textbox", outputs="textbox").launch()

In [ ]:
#to get publicly available link to this use share=True in launch method
gr.Interface(fn=call_gemini, inputs="textbox", outputs="textbox").launch(share=True)

In [ ]:
# gradio is flexible w.r.t type of inputs and outputs

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=call_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [22]:
def stream_gemini(prompt):
    response = model.generate_content(prompt, stream=True)
    result = ""
    for chunk in response:
        result += chunk.text
        yield result

def stream_ollama(prompt):
    messages=[
        {'role':'system', 'content':system_prompt},
        {'role':'user', 'content':prompt}
    ]
    response = ollama.chat(model=ollama_model,messages=messages ,stream=True)
    result = ""
    for chunk in response:
        result += chunk["message"]['content']
        yield result

    

In [ ]:
# gradio automatically infers that the function is a generator here

system_message = "You are a helpful assistant that responds in markdown"

view = gr.Interface(
    fn=stream_gemini,
    inputs=[gr.Textbox(label="Your message:", lines=6)],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

In [26]:
def stream_model(model, prompt):
    if model.upper()=="GEMINI":
       result = stream_gemini(prompt)
    elif model.upper()=="LAMA":
       result = stream_ollama(prompt)
    else:
        raise ValueError("unknown model!")
    yield from result

In [28]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Dropdown(["Gemini", "Lama"], label="Select model", value="Gemini"), gr.Textbox(label="Your message:")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
# view.launch()
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7866
* Running on public URL: https://0d79d8a85e74f1b6f7.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
